In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

tsr564_financial_news_path = kagglehub.dataset_download('tsr564/financial-news')
tsr564_financial_records_path = kagglehub.dataset_download('tsr564/financial-records')

print('Data source import complete.')


In [ ]:
!git clone https://github.com/Pritam3355/Datasets.git

fatal: destination path 'Datasets' already exists and is not an empty directory.


# Prepare DataSet

    Fundamental_Data.csv : contains P&L, balance-sheet,cash-flow,quarterly results (numerical)
    IndianFinancialNews.csv : contains news title & description (text)

Since past news/events will have less impact on current year financial data , so we take the current year news & financial data to find relation or influencing factor

In [ ]:
import pandas as pd


df1 = pd.read_csv("/kaggle/working/Datasets/Fundamental_Data.csv")
df1["year"].unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2012])

### Extract Year,Month from Date

In [ ]:
from datetime import datetime


df2 = pd.read_csv("/kaggle/working/Datasets/IndianFinancialNews.csv")
df2.drop("Unnamed: 0", axis=1, inplace=True)

years = []
months = []

month_to_number = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

for _, row in df2.iterrows():
    date_str = row['Date']
    date_list = date_str.split(",")
    month_day = date_list[0].split(" ")
    month = month_day[0]
    day = month_day[1]
    year = int(date_list[1])
    months.append(month_to_number[month])
    years.append(year)

df2["Year"] = years
df2["Month"] = months
df2.drop("Date", axis=1, inplace=True)

df2.head()

,Title,Description,Year,Month
0,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,5
1,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,5
2,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,5
3,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,5
4,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,5


### For computational constrain we only take 3 consecutive years in consideration - [2016, 2017, 2018]

In [ ]:
intersection_years = list(set(df2['Year'].unique()) & set(df1['year'].unique()))[:3]
df1 = df1[df1['year'].isin(intersection_years)]
df2 = df2[df2['Year'].isin(intersection_years)]

In [ ]:
df1.shape,df2.shape,intersection_years

((4056, 6), (6432, 4), [2016, 2017, 2018])

In [ ]:
df1.to_csv("financial_report.csv",index=False)
df2.to_csv("financial_news.csv",index=False)

In [ ]:
!pip install --q --upgrade langchain langchain-community  faiss-cpu langchain-core
!pip install --q --upgrade  langchain-text-splitters rank_bm25
!pip install -q sentence_transformers llama-index
!pip install -q --upgrade together langchain-together

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.3.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.3 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is inco

In [ ]:
import os

# from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
# os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HUGGINGFACE_API_KEY')
together_api_key = ""

### Create Emebedding & store them on FAISS VectorDB

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from tqdm import tqdm

loader1 = CSVLoader(file_path='/kaggle/working/financial_report.csv')
data1 = loader1.load()

hfembeddings1 = HuggingFaceEmbeddings(model_name='thenlper/gte-large',model_kwargs={'device': 'cpu'})

db1 = FAISS.from_documents(data1, hfembeddings1)

with tqdm(total=len(data1), desc="save FAISS Index") as pbar:
    db1.save_local('financial-records')
    pbar.update(len(data1))

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Saving FAISS Index: 100%|██████████| 4056/4056 [00:00<00:00, 115088.91it/s]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader2 = CSVLoader(file_path='/kaggle/working/financial_news.csv')
data2 = loader.load()
hfembeddings2 = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5", model_kwargs={'device': 'cpu'})

txt_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)

inp_txt = txt_splitter.split_documents(data2)

db2 = FAISS.from_documents(inp_txt, hfembeddings2)

with tqdm(total=len(data2), desc="save FAISS Index") as pbar:
    db2.save_local('financial-news')
    pbar.update(len(data2))

Saving FAISS Index: 100%|██████████| 6432/6432 [00:00<00:00, 130716.98it/s]


### Creating Ensembel retriver

**BM25 (Best Match 25) algorithm**: a ranking function used by search engines to determine the relevance of documents to a search query

**Embedding model** : Here for numerical type data we're using ***thenlper/gte-large*** model and for text type data ***BAAI/bge-base-en-v1.5***

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

faiss_retriver_1 = db1.as_retriever(search_kwargs={'k': 1})
faiss_retriever_2 = db2.as_retriever(search_kwargs={'k': 2})
bm25_retriever = BM25Retriever.from_documents(inp_txt)

# create ensemble retriever (FAISS + BM25)
ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriver_1,faiss_retriever_2, bm25_retriever],
                                       weights=[0.35,0.3, 0.35])



## Question-1

In [ ]:
query = "Is it good idea to buy HDFC at the end of September, 2018?"

### Here we load pre-computed FAISS Index

Since we would like to consider every piece of data relevant to our question - no need to rank them

In [ ]:

from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever

hfembeddings_1 = HuggingFaceEmbeddings(model_name='thenlper/gte-large',model_kwargs={'device': 'cpu'})
hfembeddings_2 = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5", model_kwargs={'device': 'cpu'})

db1 = FAISS.load_local("/kaggle/input/financial-records", hfembeddings_1, allow_dangerous_deserialization=True)

db2 = FAISS.load_local("/kaggle/input/financial-news", hfembeddings_2, allow_dangerous_deserialization=True)

faiss_retriver_1 = db1.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 5,
        'score_threshold': 0.8,
        'fetch_k': 10
    }
)
faiss_retriever_2 = db2.as_retriever(
    search_type="mmr",  # max marginal relevance - diversify results
    search_kwargs={
        'k': 5,  # top 5 docs
        'lambda_mult': 0.5,  # diversity vs relevance (0 = max diversity, 1 = max relevance)
        'fetch_k': 15,  # fetch more candidates before using mmr
    }
)

ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriver_1,faiss_retriever_2], weights=[0.55,0.45])



### Relevant context

In [ ]:
contexts = ensemble_retriever.invoke(query)
context = []
for ctx in contexts:
    if ctx.page_content.startswith("category: "):
        context.append(ctx.page_content)
    else:
        ctx = "category: News"+'\n'+ctx.page_content
        context.append(ctx)


for ctx in context:
    print(ctx,type(ctx))
    print("="*100)

category: Reserves
report_type: balance-sheet
stock_symbol: HDFCBANK
year: 2016
month: 3
value_type: 73798.0 <class 'str'>
category: Reserves
report_type: balance-sheet
stock_symbol: HDFCLIFE
year: 2016
month: 3
value_type: 1108.0 <class 'str'>
category: Dividend Payout %
report_type: profit-loss
stock_symbol: HDFCBANK
year: 2016
month: 3
value_type: 0.19 <class 'str'>
category: Equity Capital
report_type: balance-sheet
stock_symbol: HDFCBANK
year: 2016
month: 3
value_type: 506.0 <class 'str'>
category: Investments
report_type: balance-sheet
stock_symbol: HDFCLIFE
year: 2016
month: 3
value_type: 74144.0 <class 'str'>
category: News
Title: HDFC to list subsidiaries at opportune time, says chairman Deepak Parekh
Description: HDFC has various subsidiaries which can be listed
Year: 2017
Month: 6 <class 'str'>
category: News
Title: HDFC Bank raises fixed deposit interest rates by up to 0.6% from today
Description: The deposit of maturity 6-9 months would earn 6.75 per cent, 40 basis points 

In [ ]:


from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.llms import Together



model = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=128,
    top_k=50,
    together_api_key=together_api_key
)


template = """<s>[INST] Answer the question in a simple sentence based on following context, use the timeline data to aggregate financial report and news:
{context}

Question: {question} [/INST]
"""
prompt = ChatPromptTemplate.from_template(template)

context_combined = "\n".join(context)

chain = (
    {"context": RunnableLambda(lambda _: context_combined), "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
query = "Should I hold my position on HDFC at the end of November, 2018?"
output = chain.invoke(query)

print(output)


Based on the provided information, the decision to hold a position in HDFC should not be made solely based on the financial report and news from 2016 and 2017. However, it is noted that HDFC has been actively managing its subsidiaries and focusing on growth, which could be positive indicators. It is recommended to consider more recent financial reports and news, as well as other factors, when making investment decisions.


In [ ]:
# !pip install -q ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:

# from huggingface_hub import hf_hub_download
# from langchain.llms import CTransformers
# import os

# model_id = "TheBloke/mistral-7B-OpenOrca-GGUF"
# model_filename = "mistral-7b-openorca.Q8_0.gguf"

# # Download the model file to a temporary location
# model_path = hf_hub_download(repo_id=model_id, filename=model_filename)

# # Now load the model using CTransformers
# llm = CTransformers(
#     model=model_path,  # Path to the downloaded model file
#     model_type="llama",  # Specify the correct model type, e.g., "llama"
#     max_new_tokens=512,
#     temperature=0.3
# )

# # Test the LLM with a prompt
# output = llm("What is the capital of France?")
# print(output)



### create augmented query of the same question

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import PromptTemplate,ChatPromptTemplate

def query_rewrite(question,context):

    context_combined = "\n".join(context)
    template = """<s>[INST] Create 5 similar questions(no preamble) based on the provided Question sample and provided context,follow the current timeline(year,month):
    {context}

    Question: {question} [/INST]
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain =  prompt| model| StrOutputParser()

    result = chain.invoke({"question" : question,"context": context_combined})
    return result

In [ ]:
query = "Should I hold my position on HDFC at the end of November, 2018?"
augmented_queries = query_rewrite(query,context)
augmented_queries = augmented_queries.strip().split("\n")
augmented_queries = [s[s.find('.')+2:] if s.startswith('  ') else s for s in augmented_queries[1:]]
augmented_queries

['Given the recent increase in fixed deposit interest rates, should I consider buying HDFC stocks in June, 2017?',
 "Considering HDFC's focus on digital growth in August 2018, is it a good idea to invest in the bank?",
 "With HDFC Life stake sale completed in July 2016, should investors anticipate any changes in HDFC's share price?",
 'After HDFC']

### get relevant context for the augmented querirs

Since augmented query can cover larger context - we will ingest this larger context data

In [ ]:
docs = [ensemble_retriever.get_relevant_documents(query) for query in aug_queries]

unique_contents = set()
unique_docs = []
for sublist in docs:
    for doc in sublist:
        if doc.page_content not in unique_contents:
            unique_docs.append(doc)
            unique_contents.add(doc.page_content)
unique_contents = list(unique_contents)

#### get a relevance score of [orginal query,retrived context] pair using Cross-Encoder

Cross-Encoder is better than Bi-Encoder since it can take all the sentece(here augmented query's context0 at once
instead of taking them one by one

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pairs = []
for doc in unique_contents:
    pairs.append([query, doc])

scores = cross_encoder.predict(pairs)
scored_docs = zip(scores, unique_contents)
sorted_docs = sorted(scored_docs, reverse=True)
reranked_docs = [doc for _, doc in sorted_docs]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
ranked_context = []
for ctx in reranked_docs:
    if ctx.startswith("category:"):
        ranked_context.append(ctx)
    else:
        ctx = "category: News"+'\n'+ctx
        ranked_context.append(ctx)

for ctx in ranked_context:
    print(ctx)
    print("="*100)

category: News
Title: HDFC Mutual Fund allots Rs 7.3 billion shares to anchor investors
Description: HDFC MF is the second IPO from the HDFC group after HDFC Standard Life's maiden offering in November 2017
Year: 2018
Month: 7
category: News
Title: HDFC Q1 net up 54% to Rs 21.90 bn, gets nod for $1.5 bn external borrowings
Description: Its loan book grew by 18.5 per cent at Rs 3.71 trillion at end of June 2018 as against Rs 3.13 trillion a year ago
Year: 2018
Month: 7
category: News
Title: HDFC to raise up to Rs 130 billion through preference shares, QIP
Description: This will be the first equity raising by the country's largest pure-play mortgage lender in over a decade
Year: 2018
Month: 1
category: News
Title: HDFC Bank MD's remuneration declines to Rs 9.65cr in 2017-18
Description: The remuneration drawn by Puri and bank employees, in terms of ratio, stood at 209:1 meaning he took home 209 times more than the ...
Year: 2018
Month: 7
category: News
Title: Indian lender HDFC Bank's US

### create new retriver

Used BM25Retriver for ranking purpose since our context is already ranked (here using a ranking retriver makes sense)

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document

context_inp_txt = [Document(page_content=ctx) for ctx in context]
bm25_retriever = BM25Retriever.from_documents(context_inp_txt)

# create ensemble retriever (FAISS + BM25)
ensemble_retriever_2 = EnsembleRetriever(retrievers=[faiss_retriver_1,faiss_retriever_2, bm25_retriever],
                                       weights=[0.35,0.3, 0.35])


In [ ]:

from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.llms import Together


template = """<s>[INST] Answer the question in a simple sentence based on following context(sorted according to priority), use the timeline data to aggregate financial report and news:
{context}

Question: {question} [/INST]
"""
prompt = ChatPromptTemplate.from_template(template)

ranked_context_combined = "\n".join(ranked_context)

chain = (
    {"context": RunnableLambda(lambda _: ranked_context_combined ), "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
query = "Should I hold my position on HDFC till January, 2019 or should I exit before that ? If yes then how long. if no then when should I exit ?"
output = chain.invoke(query)

print(output)

As a financial assistant, I can provide you with information based on the data you've given, but if you're looking for specific investment advice, I would recommend consulting with a financial advisor.

From the data provided, it appears that HDFC has had a strong financial performance in 2018, with net profits and loan books growing significantly. The company has also been actively raising funds through various means such as IPOs, preference shares, and external borrowings. 

In terms of stock prices, there doesn't seem to be any clear trend that suggests you should sell your position immediately.


In [ ]:
#